In [1]:
import pandas as pd 
from transformers import AutoModelForMaskedLM ,AutoTokenizer


In [2]:
model_name = "bert-base-cased"

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForMaskedLM.from_pretrained(model_name)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

In [4]:
mask=tokenizer.mask_token

'[MASK]'

In [5]:
text=f'I want to {mask} pizza for tonight '

In [ ]:
tokens=tokenizer.tokenize(text)

In [ ]:
tokens

In [6]:
encode_input=tokenizer(text,return_tensors='pt')

In [9]:
outputs=model(**encode_input)

In [13]:
logits=outputs.logits.detach().numpy()[0]

In [15]:
logits

array([[ -7.428308 ,  -7.289476 ,  -7.4779015, ...,  -6.292881 ,
         -5.9588537,  -6.4330764],
       [ -7.9285583,  -8.263489 ,  -8.044237 , ...,  -6.675229 ,
         -6.4446354,  -6.89106  ],
       [-12.34469  , -11.996108 , -12.744249 , ...,  -8.402964 ,
         -6.5324244,  -8.13361  ],
       ...,
       [ -9.135826 ,  -8.89547  ,  -8.983269 , ...,  -7.861002 ,
         -5.0709443,  -8.330008 ],
       [ -9.468294 ,  -9.507522 ,  -9.067586 , ...,  -6.7673492,
         -6.186506 ,  -7.4156284],
       [-14.433859 , -14.620809 , -14.955017 , ..., -11.640886 ,
        -11.348209 , -13.298972 ]], dtype=float32)

In [19]:
mask_logits=logits[tokens.index(mask)+1]

In [20]:
from scipy.special import softmax

In [21]:
confidence_score=softmax(mask_logits)

In [22]:
import numpy as np

In [23]:
for i in np.argsort(confidence_score)[::-1][:5]:
    pred=tokenizer.decode(i)
    score=confidence_score[i]
    print(text.replace(mask,pred),score)

I want to have pizza for tonight  0.25416413
I want to eat pizza for tonight  0.17271437
I want to get pizza for tonight  0.15204938
I want to make pizza for tonight  0.11082354
I want to order pizza for tonight  0.08149521
